## Install the required dependencies

Tensorflow model maker, doesn't work on MacOS

In [1]:
%pip install -q tflite-model-maker
%pip install -q pycocotools

ERROR: Cannot install tflite-model-maker==0.1.0, tflite-model-maker==0.1.1, tflite-model-maker==0.1.2, tflite-model-maker==0.2.0, tflite-model-maker==0.2.1, tflite-model-maker==0.2.2, tflite-model-maker==0.2.3, tflite-model-maker==0.2.4, tflite-model-maker==0.2.5, tflite-model-maker==0.3.0, tflite-model-maker==0.3.1, tflite-model-maker==0.3.2, tflite-model-maker==0.3.3 and tflite-model-maker==0.3.4 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/user_guide/#fixing-conflicting-dependencies


In [2]:
# can also try nightly builds of TensorFlow
%pip install tflite-model-maker-nightly
%pip install -q pycocotools

     |████████████████████████████████| 617 kB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 508 kB 102.4 MB/s eta 0:00:01
     |████████████████████████████████| 203 kB 115.7 MB/s eta 0:00:01
     |████████████████████████████████| 37.9 MB 73.3 MB/s eta 0:00:01
  Using cached absl_py-1.0.0-py3-none-any.whl (126 kB)
  Using cached tf_models_official-2.3.0-py2.py3-none-any.whl (840 kB)
  Using cached PyYAML-6.0-cp38-cp38-macosx_11_0_arm64.whl
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached fire-0.4.0.tar.gz (87 kB)
     |████████████████████████████████| 548 kB 154.4 MB/s eta 0:00:01
  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Using cached tensorflow_model_optimization-0.7.2-py2.py3-none-any.whl (237 kB)
     |████████████████████████████████| 617 kB 89.8 MB/s eta 0:00:01
     |████████████████████████████████| 617 kB 62.5 MB/s eta 0:00:01
     |████████████████████████████████| 617 kB 143.2 MB/s eta 0:00:01
     |███████████

## Import the required packages

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

## (Step 1) Prepare your dataset:

depending on your dataset, you may not have all the folders listed below:

* Images in `train_data` is used to train the custom object detection model.
* Images in `val_data` is used to check if the model can generalize well to new images that it hasn't seen before.
* Images in `test_data` are used to evaluate the model and are optional. You can replace any instance of `test_data` with `val_data`.

In [ ]:
train_data = object_detector.DataLoader.from_pascal_voc(
    'kb4yg/train',
    'kb4yg/train',
    ['car', 'motorcycle','person','bus','bicycle'] # can also be path to labelMap file
)

val_data = object_detector.DataLoader.from_pascal_voc(
    'kb4yg/valid',
    'kb4yg/valid',
    ['car', 'motorcycle','person','bus','bicycle']
)

test_data = object_detector.DataLoader.from_pascal_voc(
    'kb4yg/test',
    'kb4yg/test',
    ['car', 'motorcycle','person','bus','bicycle']
)

Alternative methods for loading data can be found [here](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/object_detector/DataLoader)

## (Step 2) Train the model

### Select a model architecture
This tutorial uses the EfficientDet-Lite0 model. EfficientDet-Lite[0-4] are a family of mobile/IoT-friendly object detection models derived from the [EfficientDet](https://arxiv.org/abs/1911.09070) architecture.

Here is the performance of each EfficientDet-Lite models compared to each others.

| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|--------------------|-----------|---------------|----------------------|
| EfficientDet-Lite0 | 4.4       | 146           | 25.69%               |
| EfficientDet-Lite1 | 5.8       | 259           | 30.55%               |
| EfficientDet-Lite2 | 7.2       | 396           | 33.97%               |
| EfficientDet-Lite3 | 11.4      | 716           | 37.70%               |
| EfficientDet-Lite4 | 19.9      | 1886          | 41.96%               |

<i> * Size of the integer quantized models. <br/>
** Latency measured on Raspberry Pi 4 using 4 threads on CPU. <br/>
*** Average Precision is the mAP (mean Average Precision) on the COCO 2017 validation dataset.
</i>

In [ ]:
spec = model_spec.get('efficientdet_lite0')

### Pick training settings

* The EfficientDet-Lite0 model uses `epochs = 50` by default, which means it will go through the training dataset 50 times. You can look at the validation accuracy during training and stop early to avoid overfitting.
* Set `batch_size = 8` here so you will see that it takes 21 steps to go through the 175 images in the training dataset.
* Set `train_whole_model=True` to fine-tune the whole model instead of just training the head layer to improve accuracy. The trade-off is that it may take longer to train the model.

In [ ]:
model = object_detector.create(train_data, model_spec=spec, batch_size=8, train_whole_model=True, validation_data=val_data)

## (Step 3) Evaluate the model

In [ ]:
model.evaluate(test_data)

## (Step 4) Export the model

In [ ]:
model.export(export_dir='.')
# or if you want to save the list of labels
model.export(export_dir='.', export_format=[ExportFormat.SAVED_MODEL, ExportFormat.LABEL])

## (Step 5) Evaluate TF-Lite model

Several factors can affect the model accuracy when exporting to TFLite:
* [Quantization](https://www.tensorflow.org/lite/performance/model_optimization) helps shrinking the model size by 4 times at the expense of some accuracy drop.
* The original TensorFlow model uses per-class [non-max supression (NMS)](https://www.coursera.org/lecture/convolutional-neural-networks/non-max-suppression-dvrjH) for post-processing, while the TFLite model uses global NMS that's much faster but less accurate.
Keras outputs maximum 100 detections while tflite outputs maximum 25 detections.

Therefore you'll have to evaluate the exported TFLite model and compare its accuracy with the original TensorFlow model.

In [ ]:
model.evaluate_tflite('model.tflite', test_data)